In [ ]:
"""
NOTEBOOK 06: EXTRACCIÓN DE CARACTERÍSTICAS ESPACIO-TEMPORALES
==============================================================
Extraer descriptores dinámicos de la microcirculación a partir
de análisis de diagramas espacio-tiempo y perfiles de velocidad.

Características extraídas:
- Velocidad media, desviación estándar, coeficiente de variación
- Patrón de flujo (continuo/pulsátil)
- Análisis de bifurcaciones y ramificación
- Complejidad topológica

Entrada: Frames segmentados, velocidades individuales (notebooks 03-05)
Salida: DataFrame con características por paciente/visita

Autor: Tesis Doctoral - Análisis Automatizado de Dinámicas Microvasculares
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import cv2
from scipy import signal, stats
from skimage import transform

# Configuración de visualización
sns.set_style("whitegrid")
sns.set_palette("Set2")
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 13
plt.rcParams['legend.fontsize'] = 10

In [ ]:
# Save spatiotemporal features
output_path = Path('/Users/luisestebanbaldasseroni/LuisEsteban/tesis/microcirculation-analysis/src/data/spatiotemporal_features.csv')
df_spacetime.to_csv(output_path, index=False)

print(f"\nSpatiotemporal features saved to: {output_path}")
print(f"\nFeature statistics:")
print(df_spacetime[['mean_velocity', 'std_velocity', 'velocity_cv', 'num_vessels_measured']].describe().round(2))

In [ ]:
# Visualize spatiotemporal features
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Spatiotemporal Features Analysis', fontsize=14, fontweight='bold')

# Mean velocity by patient
ax = axes[0, 0]
patients = df_spacetime['patient'].unique()
for patient in patients:
    patient_data = df_spacetime[df_spacetime['patient'] == patient]
    ax.scatter(patient_data.index, patient_data['mean_velocity'], label=patient, s=100, alpha=0.7)
ax.set_ylabel('Mean Velocity')
ax.set_title('Mean Vessel Velocity by Record')
ax.legend(fontsize=8)
ax.grid(True, alpha=0.3)

# Velocity coefficient of variation
ax = axes[0, 1]
bp = ax.boxplot([df_spacetime[df_spacetime['patient'] == p]['velocity_cv'].values 
                  for p in patients], labels=patients, patch_artist=True)
for patch in bp['boxes']:
    patch.set_facecolor('lightblue')
ax.set_ylabel('Coefficient of Variation')
ax.set_title('Velocity Variability (CV) by Patient')
ax.grid(True, alpha=0.3, axis='y')

# Number of vessels measured
ax = axes[1, 0]
df_spacetime.boxplot(column='num_vessels_measured', by='patient', ax=ax)
ax.set_ylabel('Number of Vessels')
ax.set_xlabel('Patient')
ax.set_title('Vessels Measured per Record')
plt.sca(ax)
plt.xticks(rotation=45)

# Correlation between density and velocity
ax = axes[1, 1]
ax.scatter(df_spacetime['vessel_density'], df_spacetime['mean_velocity'], s=100, alpha=0.6, color='steelblue')
z = np.polyfit(df_spacetime['vessel_density'], df_spacetime['mean_velocity'], 1)
p = np.poly1d(z)
x_line = np.linspace(df_spacetime['vessel_density'].min(), df_spacetime['vessel_density'].max(), 100)
ax.plot(x_line, p(x_line), "r--", linewidth=2, label='Trend')
ax.set_xlabel('Vessel Density (TVD)')
ax.set_ylabel('Mean Velocity')
ax.set_title('Density vs Velocity Relationship')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('/Users/luisestebanbaldasseroni/LuisEsteban/tesis/microcirculation-analysis/src/data/spatiotemporal_features.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# Load integration data
integration_path = Path('/Users/luisestebanbaldasseroni/LuisEsteban/tesis/microcirculation-analysis/src/data/segmentation_annotation_integration.csv')
df_integration = pd.read_csv(integration_path)

# Load original annotations for velocity data
annotations_path = Path('/Users/luisestebanbaldasseroni/LuisEsteban/tesis/microcirculation-analysis/src/data/annotations_processed.csv')
df_annotations = pd.read_csv(annotations_path)

# Extract velocity columns
velocity_cols = [col for col in df_annotations.columns if col.startswith('velocity_')]

print(f"Processing {len(velocity_cols)} individual vessel velocity measurements")

# Compute spatiotemporal features for each patient-visit
spatiotemporal_features = []

for idx, row in df_integration.iterrows():
    record_id = row['record_id']
    
    # Find matching annotation row
    annotation_row = df_annotations[df_annotations['record_id'] == record_id]
    
    if len(annotation_row) > 0:
        annotation_row = annotation_row.iloc[0]
        
        # Extract velocity measurements
        velocities = [annotation_row[col] for col in velocity_cols 
                     if not pd.isna(annotation_row[col]) and annotation_row[col] > 0]
        
        if velocities:
            velocities = np.array(velocities)
            
            # Compute spatiotemporal features
            features = {
                'record_id': record_id,
                'patient': row['patient'],
                'visit': row['visit'],
                'mean_velocity': np.mean(velocities),
                'std_velocity': np.std(velocities),
                'min_velocity': np.min(velocities),
                'max_velocity': np.max(velocities),
                'velocity_range': np.max(velocities) - np.min(velocities),
                'num_vessels_measured': len(velocities),
                'velocity_cv': np.std(velocities) / np.mean(velocities) if np.mean(velocities) > 0 else 0,
                'vessel_density': row['annotation_tvd'] if 'annotation_tvd' in row else 0
            }
            
            spatiotemporal_features.append(features)

df_spacetime = pd.DataFrame(spatiotemporal_features)

print(f"\nExtracted spatiotemporal features for {len(df_spacetime)} records")
print("\nFeature Summary:")
print(df_spacetime[['patient', 'mean_velocity', 'std_velocity', 'velocity_cv', 'num_vessels_measured']].head(10))

# Notebook 06: Space-Time Feature Extraction

Extracts spatiotemporal features from vessel dynamics including flow velocity, acceleration, and temporal coherence.

# 06 - Extracción de métricas fisiológicas

## Objetivo
Calcular:
- Velocidad de eritrocitos
- Densidad lineal
- Tasa de suministro


In [ ]:
from src.features.compute_metrics import extract_metrics

metrics = extract_metrics(space_time_img)
print(metrics)

# {'velocity': 115.3, 'density': 78.2, 'supply_rate': 44.6}
